# ComfyUI on Google Colab with Your Google Drive Models

This notebook will:
1. Install ComfyUI
2. Mount your Google Drive
3. Link your backed-up models from Google Drive
4. Start ComfyUI with Cloudflare tunnel for web access

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Check if ComfyUI models exist in Google Drive
!ls -la /content/drive/MyDrive/ComfyUI/models/

In [ ]:
# Install ComfyUI and dependencies
%cd /content
!git clone https://github.com/comfyanonymous/ComfyUI
%cd ComfyUI
!pip install -r requirements.txt
!pip install cloudflared

In [ ]:
# Create symbolic links to your Google Drive models
# This avoids copying large files and saves Colab disk space

import os
import shutil

# Remove default models directory
if os.path.exists('/content/ComfyUI/models'):
    shutil.rmtree('/content/ComfyUI/models')

# Create symbolic link to Google Drive models
!ln -s /content/drive/MyDrive/ComfyUI/models /content/ComfyUI/models

# Verify the link
!ls -la /content/ComfyUI/models/

In [ ]:
# Optional: Link only specific model types to save memory
# Uncomment and modify as needed

# # Create models directory
# !mkdir -p /content/ComfyUI/models

# # Link only specific subdirectories
# !ln -s /content/drive/MyDrive/ComfyUI/models/checkpoints /content/ComfyUI/models/checkpoints
# !ln -s /content/drive/MyDrive/ComfyUI/models/loras /content/ComfyUI/models/loras
# !ln -s /content/drive/MyDrive/ComfyUI/models/vae /content/ComfyUI/models/vae
# !ln -s /content/drive/MyDrive/ComfyUI/models/clip_vision /content/ComfyUI/models/clip_vision

In [ ]:
# Start ComfyUI with Cloudflare tunnel
import subprocess
import threading
import time
import socket

def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()

    print("\nComfyUI finished loading!")

# Start ComfyUI
threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

# Start cloudflared tunnel
!nohup cloudflared tunnel --url http://localhost:8188 > cloudflared.log 2>&1 &

# Give cloudflared time to start
time.sleep(4)

# Display the tunnel URL
print("\n" + "="*50)
print("Your ComfyUI URL:")
!grep -o 'https://.*\.trycloudflare.com' cloudflared.log | head -1
print("="*50 + "\n")

# Run ComfyUI
!python main.py --dont-print-server

## Tips for using ComfyUI on Colab:

1. **GPU Selection**: Go to Runtime > Change runtime type > T4 GPU (free) or better

2. **Model Loading**: Since models are on Google Drive, initial loading might be slower than local SSD

3. **Selective Model Loading**: If you run into memory issues, modify the notebook to only link specific model folders you need

4. **Saving Outputs**: Outputs are saved to `/content/ComfyUI/output/`. You may want to copy important outputs to Google Drive before the session ends:
   ```python
   !cp -r /content/ComfyUI/output/* /content/drive/MyDrive/ComfyUI/outputs/
   ```

5. **Custom Nodes**: Install any custom nodes you need in additional cells